In [1]:
import json
import numpy as np
import pandas as pd
import uproot
import matplotlib.pyplot as plt
import mplhep as mh

from hist.hist import Hist
from hist.axis import Regular

from pathlib import Path

from typing import Optional, Union

In [2]:
from functools import cache
from functools import cached_property

@cache
def get_segment(ring: int, station: int, sector: int, subsector: int) -> int:
    """
    https://github.com/cms-sw/cmssw/blob/CMSSW_13_3_0_pre3/Geometry/RPCGeometry/src/RPCGeomServ.cc#L361-L368
    """
    nsub = 3 if ring == 1 and station > 1 else 6
    return subsector + nsub * (sector - 1)


@cache
def get_roll_name(region: int, ring: int, station: int, sector: int, layer: int,
             subsector: int, roll: int
) -> str:
    """
    https://github.com/cms-sw/cmssw/blob/CMSSW_13_3_0_pre3/Geometry/RPCGeometry/src/RPCGeomServ.cc#L11-L87
    """
    if region == 0:
        name = f'W{ring:+d}_RB{station}'

        if station <= 2:
            name += 'in' if layer == 1 else 'out'
        else:
            if sector == 4 and station == 4:
                name += ['--', '-', '+', '++'][subsector - 1]
            elif (station == 3) or (station == 4 and sector not in (4, 9, 11)):
                name += '-' if subsector == 1 else '+'
        name += f'_S{sector:0>2d}_'
        name += ['Backward', 'Middle', 'Forward'][roll - 1]
    else:
        segment = get_segment(ring, station, sector, subsector)
        name = f'RE{station * region:+d}_R{ring}_CH{segment:0>2d}_'
        name += ['A', 'B', 'C', 'D', 'E'][roll - 1]
    return name

In [3]:
working_dir = Path('/home/eigen1907/GoogleDrive/Workspace-RPC/240425-TnP_RPC24/TnP_Plotting')

input_path = working_dir / 'data' / 'Muon__Run2022C.root'
geom_path = working_dir / 'geometry' / 'run3.csv'
roll_blacklist_path = working_dir / 'blacklist' / 'roll-blacklist.json'

data = uproot.open(f"{str(input_path)}:tree").arrays(library='np')

geom = pd.read_csv(geom_path)

if roll_blacklist_path is None:
    roll_blacklist = set()
else:
    with open(roll_blacklist_path) as stream:
        roll_blacklist = set(json.load(stream))

In [ ]:
is_fiducial = data['is_fiducial']
fiducial_data = {}

for key, values in data.items():
    fiducial_data[key] = data[key][is_fiducial]

print(len(data['is_matched']))
print(len(fiducial_data['is_matched']))
print(fiducial_data)

11712336
8565421


In [ ]:
fiducial_data['roll_name'] = [get_roll_name(
    fiducial_data['region'][idx], fiducial_data['ring'][idx], fiducial_data['station'][idx],
    fiducial_data['sector'][idx], fiducial_data['layer'][idx], fiducial_data['subsector'][idx], fiducial_data['roll'][idx]) 
    for idx in range(len(fiducial_data['region']))]

print(fiducial_data['roll_name'])